In [1]:
import urllib
from urllib import request
from bs4 import BeautifulSoup
import os
import requests
import argparse
import re
import time
import json
import math
import yake
import pickle as pkl
import numpy as np
import pandas as pd
import multiprocessing as mp
from functools import partial

import codecs
import urllib.parse as up
from crawler import get_paper_info

# Crawler and Semantic Scholar Information Extractor

In [2]:
# Generate a concatenated tldr string of papers from a certain query. Also return the information pack of these papers.
def SSSQuery(query, num_item=50, offset=0, fos=None):
    # Send a Semantic API post to get the result paper list
    query = '+'.join(query.split())
    # query_num = int(num_item*1.1)
    url = f'https://api.semanticscholar.org/graph/v1/paper/search?query={query}&offset={offset}&limit={num_item}&fields=fieldsOfStudy,abstract'
    print(url)
    paper_list = str(request.urlopen(url).read(), 'utf-8')
    paper_list = json.loads(paper_list)

    tldr_cat = ''
    info_pack = []
    # Loop through the paper list and get information pack for each paper
    for paper in paper_list['data']:
        paper_id = paper['paperId']
        # Each paper info pack consists title, abstract, authors, tldr, citation count, field of study
        paper_url = f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=title,abstract,citations.authors,tldr,citationCount,influentialCitationCount,fieldsOfStudy'
        paper_info = str(request.urlopen(paper_url).read(), 'utf-8')
        paper_info = json.loads(paper_info)
        # Filter out the papers out of field of study
        if fos is not None and fos not in paper_info['fieldsOfStudy']:
            continue
        tldr = paper_info['tldr']
        if tldr is not None:
            tldr_cat += ' '+tldr['text'].strip()
            info_pack.append(paper_info)
    tldr_cat = tldr_cat.strip()
    return tldr_cat, info_pack

In [3]:
def multi_map(func, data, processes=None):
    if processes is None:
        processes = mp.cpu_count()
    with mp.Pool(processes) as pool:
        results = pool.map(func, data)
    return results

def crawl_one(url):
    try:
        paper_batch = str(request.urlopen(url).read(), 'utf-8')
    except Exception as e:
        with open('log.txt', 'a+') as f:
            print(f'[ERROR]: {str(e)}')
            f.writelines(str(e)+f': {url}\n')
        time.sleep(360)
        crawl_one(url)

# Generate a concatenated tldr string of papers from a certain query. Also return the information pack of these papers.
def SSSQueryMulti(query, num_item=50, offset=0, fos=None):
    global info_pack
    # Send a Semantic API post to get the result paper list
    query = '+'.join(query.split())
    get_paper_info_fos = partial(get_paper_info, fos=fos)

    # query_num = int(num_item*1.1)
    if num_item <= 98:
        url = f'https://api.semanticscholar.org/graph/v1/paper/search?query={query}&offset={offset}&limit={num_item}&fields=fieldsOfStudy,abstract'
        print(url)
        paper_list = str(request.urlopen(url).read(), 'utf-8')
        paper_list = json.loads(paper_list)['data']
        info_pack = multi_map(get_paper_info_fos, paper_list)
    else:
        query_idx = 0
        error_count = 0
        # paper_list = []
        info_pack = []
        while query_idx < num_item:
            limit = 98 if query_idx + 98 < num_item else num_item - query_idx
            url = f'https://api.semanticscholar.org/graph/v1/paper/search?query={query}&offset={query_idx+offset}&limit={limit}&fields=fieldsOfStudy,abstract'
            print(url)
            try:
                paper_batch = str(request.urlopen(url).read(), 'utf-8')
            except Exception as e:
                with open('log.txt', 'a+') as f:
                    print(f'[ERROR]: {str(e)}')
                    f.writelines(str(e)+f': {url}\n')
                error_count += 1
                if error_count <= 5:
                    print(f'ERROR. Start sleeping for 6 min...')
                    time.sleep(360)
                    continue
                else:
                    break
            paper_batch = json.loads(paper_batch)['data']
            # paper_list += paper_batch
            query_idx += 98
            if paper_batch is not None:
                info_pack += multi_map(get_paper_info_fos, paper_batch)
            
            # time.sleep(360)

    info_pack = [i for i in info_pack if i is not None]
    print(f"Number of papers: {len(info_pack)}")
    # tldr_cat = ' '.join([i['tldr']['text'] for i in info_pack])
    return info_pack #tldr_cat, 

In [4]:
# kws = ['transformer', 'attention']
info_pack = []
info_pack_all_kws = []
kws = pd.read_csv(r'./data/keyword_rest.csv', header=None)
kws = [i.lower() for i in kws[0]]

In [9]:
# try:
# info_pack_all_kws = []
for kw in kws:
    info_pack = SSSQueryMulti(kw, num_item=294, fos='Computer Science')
    info_pack_all_kws += info_pack
print(len(info_pack_all_kws))
# except Exception as e:
#     with open('log.txt', 'a+') as f:
#             f.writelines(str(e))

https://api.semanticscholar.org/graph/v1/paper/search?query=ethics+and+nlp&offset=0&limit=98&fields=fieldsOfStudy,abstract
Done. Start sleeping for 6 min...
https://api.semanticscholar.org/graph/v1/paper/search?query=ethics+and+nlp&offset=98&limit=98&fields=fieldsOfStudy,abstract
Done. Start sleeping for 6 min...
https://api.semanticscholar.org/graph/v1/paper/search?query=ethics+and+nlp&offset=196&limit=98&fields=fieldsOfStudy,abstract
[ERROR]: HTTP Error 500: Internal Server Error
https://api.semanticscholar.org/graph/v1/paper/search?query=ethics+and+nlp&offset=196&limit=98&fields=fieldsOfStudy,abstract
[ERROR]: HTTP Error 500: Internal Server Error
https://api.semanticscholar.org/graph/v1/paper/search?query=ethics+and+nlp&offset=196&limit=98&fields=fieldsOfStudy,abstract
[ERROR]: HTTP Error 500: Internal Server Error
https://api.semanticscholar.org/graph/v1/paper/search?query=ethics+and+nlp&offset=196&limit=98&fields=fieldsOfStudy,abstract
[ERROR]: HTTP Error 500: Internal Server Err

In [10]:
print(len( [i['tldr']['text'] for i in info_pack_all_kws]))
print(len([np.max((np.log10(i['citationCount']+0.01),0)) for i in info_pack]))
print(len([np.max((np.log10(i['influentialCitationCount']+0.01),0)) for i in info_pack]))

1219
272
272


In [14]:
from mlib import mio
info_top = mio.load('data/transformer_citation_top5.pkl')

In [15]:
info_backup = info_pack_all_kws

info_real_all = info_top + info_pack_all_kws
info_real_all_set = list({v['tldr']['text']:v for v in info_real_all}.values())
print(len(info_top), len(info_real_all), len(info_backup), len(info_real_all_set))

1219 7551 6332


In [38]:
stem = 'transformer_citation_all_set'
ip = info_real_all_set
df = pd.DataFrame({'text': [i['tldr']['text'] for i in ip], 'citationCount': [np.max((np.log10(i['citationCount']+0.01),0)) for i in ip], 'influentialCitationCount':[np.max((np.log10(i['influentialCitationCount']+0.01),0)) for i in ip]})
df.to_csv(f'data/{stem}.csv', index=None)
with open(f'data/{stem}.pkl', 'wb') as f:
    pkl.dump(ip, f)
df.head()

,text,citationCount,influentialCitationCount
0,The term computational social science of disas...,0.778874,0.000000
1,"Wearable healthcare data sensors, global posit...",0.000000,0.000000
2,The extent to which big data analytics and dat...,1.230704,0.000000
3,The emergence of a computational social scienc...,0.699838,0.000000
4,It is argued against the assertion that theory...,2.421620,0.845718


In [40]:
stem = 'transformer_citation_filtered_set'
info_pack_valid = [i for i in info_real_all_set if i['citationCount']>1]
print(len(info_pack_valid))
ip = info_pack_valid
df = pd.DataFrame({'text': [i['tldr']['text'] for i in ip], 'citationCount': [np.max((np.log10(i['citationCount']+0.01),0)) for i in ip], 'influentialCitationCount':[np.max((np.log10(i['influentialCitationCount']+0.01),0)) for i in ip]})
df.to_csv(f'data/{stem}.csv', index=None)
with open(f'data/{stem}.pkl', 'wb') as f:
    pkl.dump(ip, f)
df.head()

4951


,text,citationCount,influentialCitationCount
0,The term computational social science of disas...,0.778874,0.000000
1,The extent to which big data analytics and dat...,1.230704,0.000000
2,The emergence of a computational social scienc...,0.699838,0.000000
3,It is argued against the assertion that theory...,2.421620,0.845718
4,This paper argues for a comprehensive and bala...,0.778874,0.000000


In [26]:
# df_valid = df[df.citationCount.ge(0.1)]
# with open(f'data/{stem}.pkl', 'wb') as f:
#     pkl.dump(info_pack_all_kws, f)

# Sketch

In [31]:
tldr_cat, info_pack = SSSQuery('transformer', num_item=100, fos='Computer Science')
print([i['citationCount'] for i in info_pack])
print([i['influentialCitationCount'] for i in info_pack])

https://api.semanticscholar.org/graph/v1/paper/search?query=transformer&offset=0&limit=100&fields=fieldsOfStudy,abstract
[4114, 2267, 1421, 533, 493, 320, 148, 185, 187, 138, 124, 127, 116, 356, 240, 172, 102, 80, 68, 231, 185, 167, 161, 394, 167, 282, 299, 176, 144, 52, 219, 124, 143, 182, 102, 107, 446, 92, 287, 134, 77, 67, 61, 58, 57, 55, 62, 44, 78, 80, 108, 130, 92, 62, 168, 153, 95, 92, 45, 88, 39, 77, 62, 60, 141, 111, 93, 256, 177]
[527, 416, 183, 131, 72, 63, 39, 50, 31, 27, 9, 16, 9, 70, 42, 43, 6, 24, 9, 30, 29, 31, 29, 124, 27, 25, 20, 7, 29, 2, 42, 33, 21, 53, 7, 14, 26, 7, 26, 43, 11, 3, 8, 11, 3, 3, 1, 7, 25, 14, 15, 10, 15, 6, 10, 8, 8, 8, 8, 13, 5, 16, 1, 1, 7, 18, 5, 16, 4]


In [10]:
info_pack[0]['tldr']['text']

'This work introduces a new learnable module, the Spatial Transformer, which explicitly allows the spatial manipulation of data within the network, and can be inserted into existing convolutional architectures, giving neural networks the ability to actively spatially transform feature maps.'

In [34]:
np.max((np.log10(0), 0))

C:\Users\miracleyoo\Anaconda3\envs\cs685\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


0.0

In [35]:
df = pd.DataFrame({'text': [i['tldr']['text'] for i in info_pack], 'citationCount': [np.max((np.log10(i['citationCount']+0.01),0)) for i in info_pack], 'influentialCitationCount':[np.max((np.log10(i['influentialCitationCount']+0.01),0)) for i in info_pack]})
df.to_csv('transformer_citation.csv', index=None)
df.head()

,text,citationCount,influentialCitationCount
0,"A new simple network architecture, the Transfo...",4.417571,3.773421
1,A global approach which always attends to all ...,3.734320,2.742733
2,An attention based model that automatically le...,3.826464,2.769385
3,The proposed SAGAN achieves the state-of-the-a...,3.234520,2.287824
4,New state-of-the-art segmentation performance ...,3.157762,2.206853


In [5]:
tldr_cat

'This work introduces a new learnable module, the Spatial Transformer, which explicitly allows the spatial manipulation of data within the network, and can be inserted into existing convolutional architectures, giving neural networks the ability to actively spatially transform feature maps. This systematic study compares pre-training objectives, architectures, unlabeled datasets, transfer approaches, and other factors on dozens of language understanding tasks and achieves state-of-the-art results on many benchmarks covering summarization, question answering, text classification, and more. This work proposes a novel neural architecture Transformer-XL that enables learning dependency beyond a fixed length without disrupting temporal coherence, which consists of a segment-level recurrence mechanism and a novel positional encoding scheme. Following prior work on long-sequence transformers, the Longformer is evaluated on character-level language modeling and achieves state-of-the-art result

# Keywords Extraction

In [6]:
language = "en"
max_ngram_size = 1
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(tldr_cat)
keywords = sorted(keywords, key=lambda x:x[1])
for kw in keywords:
    print(kw)

('Spatial', 0.03213113441634914)
('work', 0.04798022059872212)
('language', 0.06973861700167353)
('Transformer', 0.07994217878795795)
('tasks', 0.08425923835568348)
('architectures', 0.12034394262796394)
('Longformer', 0.1231255301810972)
('neural', 0.13795581464104506)
('training', 0.13840506098154778)
('module', 0.14654904914113015)
('giving', 0.14654904914113015)
('maps', 0.14654904914113015)
('achieves', 0.1514314025909243)
('results', 0.1514314025909243)
('introduces', 0.15751851687783422)
('learnable', 0.15751851687783422)
('explicitly', 0.15751851687783422)
('manipulation', 0.15751851687783422)
('data', 0.15751851687783422)
('inserted', 0.15751851687783422)


In [19]:
from keybert import KeyBERT

# doc = """
#          Supervised learning is the machine learning task of learning a function that
#          maps an input to an output based on example input-output pairs. It infers a
#          function from labeled training data consisting of a set of training examples.
#          In supervised learning, each example is a pair consisting of an input object
#          (typically a vector) and a desired output value (also called the supervisory signal). 
#          A supervised learning algorithm analyzes the training data and produces an inferred function, 
#          which can be used for mapping new examples. An optimal scenario will allow for the 
#          algorithm to correctly determine the class labels for unseen instances. This requires 
#          the learning algorithm to generalize from the training data to unseen situations in a 
#          'reasonable' way (see inductive bias).
#       """
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(tldr_cat, keyphrase_ngram_range=(1,3), stop_words='english', top_n = 10, nr_candidates=40, use_mmr=True, diversity=0.7)
print(keywords)

[('neural architecture transformer', 0.5527), ('length disrupting', 0.1076), ('benchmarks covering summarization', 0.2611), ('language showed newly', 0.2474), ('explicitly allows spatial', 0.2433), ('replaces dot product', 0.2065), ('answering', 0.2506), ('finetune variety downstream', 0.0994), ('sampling algorithm hgsampling', -0.0439), ('nlp tasks layerdrop', 0.323)]


# Development Scratch

In [21]:
num_item = 30
offset = 0
query = 'hand'
query = '+'.join(query.split())
url = f'https://api.semanticscholar.org/graph/v1/paper/search?fos[0]=computer-science&query={query}&offset={offset}&limit={num_item}'
paper_list = str(request.urlopen(url).read(), 'utf-8')
paper_list = json.loads(paper_list)
# print(paper_list)

In [42]:
for paper in paper_list['data']:
    paper_id = paper['paperId']
    paper_url = f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=citations.authors,tldr,citationCount,fieldsOfStudy'
    paper_info = str(request.urlopen(paper_url).read(), 'utf-8')
    paper_info = json.loads(paper_info)
    print(paper_info['citationCount'], paper_info['fieldsOfStudy'])